In [0]:
#Download and unzip files
!pip3 install scikit-learn
!sudo apt-get install unzip
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
--2020-05-04 01:07:59--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.94.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.94.253|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
#Verfiy file download
#models.py
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
import itertools
import csv
import pandas as pd
import spacy
from torchtext.vocab import GloVe
import re
from torchtext import data
import pickle
import torch.optim as optim
import gensim
import torchtext.vocab as vocab
from tqdm import tqdm_notebook
#Verify CUDA acceleration should print cuda:0
print(device)

cuda:0


In [0]:
from torchtext.data import Field
import spacy
from gensim.models import KeyedVectors

# Train Word Embeddings and save
 
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
model_em = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
weights = torch.FloatTensor(model_em.vectors)
t_embedding = nn.Embedding.from_pretrained(weights)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# NLP = spacy.load('en')
MAX_CHARS = 20000
MAX_VOCAB_SIZE = 25000
BATCH_SIZE = 64
comment = data.Field(
    tokenize='spacy'
)
label = data.LabelField()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def load_data(train_file, val_file, test_file, covid_file):

    train = data.TabularDataset(
        path=train_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    val = data.TabularDataset(
        path=val_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    test = data.TabularDataset(
        path=test_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    covid = data.TabularDataset(
        path=covid_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])

    
    comment.build_vocab(train,
                        max_size=MAX_VOCAB_SIZE,
                        min_freq=5,
                        #vectors=GloVe(name='6B', dim=300),
                        #unk_init=torch.Tensor.normal_
                        )
          
    label.build_vocab(train)
    print(label.vocab.stoi)

    BATCH_SIZE = 64

    train_iterator, val_iterator, test_iterator, covid_iterator = data.BucketIterator.splits(
        (train, val, test, covid),
        batch_size=BATCH_SIZE,
        device=device,
        sort=False)
    # pickle.dump(train, open("train_iter.p", "wb"))
    # pickle.dump(val, open("val_iter.p", "wb"))
    # pickle.dump(test, open("test_iter.p", "wb"))
    return train_iterator, val_iterator, test_iterator, covid_iterator


class LSTMCNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx, batch_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)       
        self.dropout = nn.Dropout(dropout)
        self.lstm2 =nn.LSTM(
            input_size=embedding_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            dropout =0.5
           )

  def forward(self, text):
      # LSTM
      text = text.permute(1, 0)
      embedded = self.embedding(text)
      embedded.permute(2,1,0) #[64, 33, 300]
      embedded, __ = self.lstm2(embedded) # (seq_len, batch, num_directions * hidden_size)
      # CNN
      embedded = embedded.unsqueeze(1)# [64, 1, 33, 300]
      conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
      pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
      cat = self.dropout(torch.cat(pooled, dim=1))
      return self.fc(cat)

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')


In [0]:
# combined model
class CNNLSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx, batch_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)       
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size=64,
            num_layers=1)
        
        self.hidden2label1 = nn.Linear(64,64//2)
        self.hidden2label2 = nn.Linear(64//2,output_dim)

  def forward(self, text):
      embedded = self.embedding(text)  
      embedded = self.dropout(embedded)           
      embedded = embedded.permute(1,0,2) 
      embedded = embedded.unsqueeze(1)      
      conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]            
      cnn_out = torch.cat(conved,0)
      cnn_out = torch.transpose(cnn_out,1,2)
      
      #LSTM layer
      # [64, 28, 300]
      lstm_out, hidden1 = self.lstm(cnn_out)
      # [64, 28, 64]
      lstm_out = torch.transpose(lstm_out,1,2)
      # 64,64,28
      lstm_out = F.max_pool1d(lstm_out,lstm_out.size(2)).squeeze(2)
      #[64,64]

      # linear
      combined = self.hidden2label1(lstm_out)
      combined = self.hidden2label2(combined)
      return combined
      


In [0]:
############################################################
# Reference Code
############################################################
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]), max_preds, correct


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        preds=[]
        trues=[]
        optimizer.zero_grad()        
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc,max_preds,correct = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        # f1_score
        preds += predictions.squeeze(1).argmax(dim=1).tolist()
        trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    preds=[]
    trues=[]
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc, max_pred, correct = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # f1_score
            preds += predictions.squeeze(1).argmax(dim=1).tolist()
            trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs




In [0]:
  
  train_iterator, val_iterator, test_iterator, covid_iterator = load_data('train.csv', 'val.csv','test.csv', 'covid19.csv')
  INPUT_DIM = len(comment.vocab)#comment.vocab
  EMBEDDING_DIM = 300
  N_FILTERS = 300
  FILTER_SIZES = [5]
  OUTPUT_DIM = len(label.vocab)
  DROPOUT = 0.5
  PAD_IDX = comment.vocab.stoi[comment.pad_token]
  N_EPOCHS = 5

  model = LSTMCNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX, BATCH_SIZE)
  model.embedding = t_embedding
  #UNK_IDX = comment.vocab.stoi[comment.unk_token]
  #model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
  #model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
 
  optimizer = optim.Adadelta(model.parameters(), weight_decay=1e-5) # apply l2-reg
  criterion = nn.CrossEntropyLoss()
  #model.to(torch.device('cuda'))
  model = model.to(device)
  criterion = criterion.to(device)
  best_valid_loss = float('inf')
  train_preds=[]
  train_trues=[]
  val_preds =[]
  val_trues =[]
  for epoch in range(N_EPOCHS):

      start_time = time.time()
      train_loss, train_acc, train_pred, train_true = train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_acc, val_pred, val_true = evaluate(model, val_iterator, criterion)
      val_preds.extend(val_pred)
      val_trues.extend(val_true)
      train_preds.extend(train_pred)
      train_trues.extend(train_true)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'baseline-model.pt')

      print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')


defaultdict(<function _default_unk_index at 0x7f855e4bfb70>, {'0': 0, '1': 1, '-1': 2})


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 01 | Epoch Time: 2m 2s
	Train Loss: 0.406 | Train Acc: 85.16%
	 Val. Loss: 0.269 |  Val. Acc: 90.91%
Epoch: 02 | Epoch Time: 2m 2s
	Train Loss: 0.256 | Train Acc: 91.46%
	 Val. Loss: 0.205 |  Val. Acc: 93.25%
Epoch: 03 | Epoch Time: 2m 2s
	Train Loss: 0.216 | Train Acc: 92.99%
	 Val. Loss: 0.182 |  Val. Acc: 94.20%
Epoch: 04 | Epoch Time: 2m 2s
	Train Loss: 0.199 | Train Acc: 93.65%
	 Val. Loss: 0.161 |  Val. Acc: 94.79%
Epoch: 05 | Epoch Time: 2m 2s
	Train Loss: 0.190 | Train Acc: 94.02%
	 Val. Loss: 0.190 |  Val. Acc: 94.04%


In [0]:

test_loss, test_acc, test_preds, test_trues = evaluate(model, test_iterator, criterion)
covid_loss, covid_acc, covid_preds, covid_trues = evaluate(model, covid_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')
print(f'covid Loss: {covid_loss:.3f} | covid Acc: {covid_acc * 100:.2f}%') 
# f1 score
from sklearn.metrics import f1_score,recall_score, precision_score
#print(f1_score(preds, trues, average='weighted'))
# training 
print('training result:')
print('f1: ', f1_score(train_trues, train_preds, average='macro'))
print('precision: ',precision_score(train_trues,train_preds,average='macro'))
print('recall: ',recall_score(train_trues, train_preds,average='macro'))

# validation
print('validation result:')
print('f1: ',f1_score(val_trues, val_preds, average='macro'))
print('precision: ',precision_score(val_trues,val_preds,average='macro'))
print('recall: ',recall_score(val_trues, val_preds,average='macro'))

#test 
print('test result:')
print('f1: ',f1_score(test_trues, test_preds, average='macro'))
print('precision: ',precision_score(test_trues,test_preds,average='macro'))
print('recall: ',recall_score(test_trues, test_preds,average='macro'))

#covid 
print('covid19 result:')
print('f1: ',f1_score(covid_trues, covid_preds, average='macro'))
print('precision: ',precision_score(covid_trues,covid_preds,average='macro'))
print('recall: ',recall_score(covid_trues, covid_preds,average='macro'))
'''
0.8812620211468666 f
0.8854992768977231
0.8774410774410774
0.9104255886331983 f
0.9304217405792423
0.8945494308819626
0.9200114337537629 f
0.9372097195318804
0.9060100274986641
'''

Test Loss: 0.188 | Test Acc: 94.07%
covid Loss: 2.083 | covid Acc: 60.22%
training result:
f1:  0.9230694157111632
precision:  0.9411472697186983
recall:  0.9079503013326543
validation result:
f1:  0.9145283589090148
precision:  0.9289052139406025
recall:  0.9024539882512936
test result:
f1:  0.9246530259254291
precision:  0.9476626571033725
recall:  0.9065350803386938
covid19 result:
f1:  0.5929929573539244
precision:  0.6359748866765957
recall:  0.5821065469719222


'\n0.8812620211468666 f\n0.8854992768977231\n0.8774410774410774\n0.9104255886331983 f\n0.9304217405792423\n0.8945494308819626\n0.9200114337537629 f\n0.9372097195318804\n0.9060100274986641\n'

In [0]:
print(test_preds)


[1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 0, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 2, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 2, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 

In [0]:
import csv
from itertools import zip_longest
d = [test_preds]
export_data = zip_longest(*d, fillvalue = '')
with open('lstm_result.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("label"))
      wr.writerows(export_data)
myfile.close()



In [0]:
def label_standard(file):
    values = list(pd.read_csv(file).values.T[0])
    for i in range(len(values)):
        if values[i] == 2:
            values[i] = -1
    print(set(values))
    c_df = pd.DataFrame(values)
    c_df = c_df.rename(columns={0:'label'})
    c_df.to_csv(file,index=False)

In [50]:
label_standard("lstm_result.csv")

{0.0, 1.0, -1}
